# GIS GZ – Übung 3: Modellierung

## Einleitung
In der letzten Übung haben Sie gelernt, wie Sie Geodaten prozessieren und visualisieren kön-nen. Die heutige Übung befasst sich nicht mit Daten, sondern mit der Datenmodellierung. Aus zwei Texten leiten Sie die entsprechenden UML-Diagramme ab und setzen (sofern tech-nisch möglich) die konzeptionelle Modellierung eines Beispiels mit Fiona um. 

### Grobziele
* Sie können ein konzeptionelles Modell erstellen und es in ein logisches Datenmodell und in ein UML-Diagramm überführen. 

### Feinziele
* Sie können ein konzeptionelles Modell erstellen. 
* Sie können aus einer textuellen Aufgabenstellung ein UML-Diagramm ableiten und zeichnen. 
* Sie können ein logisches Datenmodell (so weit wie möglich) in Fiona als Schema umsetzen.

### Projekt
* Sie können das erlernte Wissen der Modellierung auf Ihr Projekt umsetzen.

## Aufgabe 1: UML-Diagramme ableiten

Überführen Sie die folgenden Beschreibungen in jeweils ein UML-Diagramm, das Klassen, Attribute und Kardinalitäten korrekt modelliert. Zum Zeichnen der Diagramme können Sie https://www.draw.io/ verwenden.

1. In unserem Verkehrsverbund gibt es Haltestellen mit einem eindeutigen Namen. Eine Haltestelle besteht aus mindestens einem Haltepunkt mit einer Adresse. An einem Haltepunkt fahren eine oder mehrere Linien. Es gibt Buslinien, Tramlinien und S-Bahn-Linien. Eine Linie hat zwei Endhaltestellen.


2. Ein Café hat einen Namen und eine Bewertung. Cafés liegen immer an einer Strasse. Es gibt Fahrtstrassen, Fusswege und Nebenstrassen. Zusätzlich gibt es Cafés, die ei-nen oder mehrere Gärten haben. Auch Gärten haben eine Betriebszeit und zusätzlich eine bestimmte Anzahl an Sitzplätzen. Betriebszeiten sind die Wintersaison, Sommer-saison oder ganzjährig. Cafés haben jeweils einen Besitzer, dessen Name, Telefon und Emailadresse bekannt sind.

## Aufgabe 2: Schema in Fiona festlegen

In dieser Aufgabe erhalten Sie die Datei `fliessgewaesser_kanton_zuerich.json`, die einen Auszug aus dem Gewässernetz des Kantons Zürich darstellt (Quelle.: `swissTLM3D` von swisstopo). Die zwei Attribute `VERLAUF` UND `STUFE` enthalten einen Code, der Aussenstehenden nichts aussagt. Daher möchten wir, dass die Bedeutung dieser Codes in zwei neue Attribute hineingeschrieben wird. Zudem wollen wir herausfinden, wie lang jedes Flusssegment ist. Diese Information möchten wir ebenfalls als separates Attribut abspeichern. 

### Verwendete libraries und Funktionen

Zuerst importieren wir die nötigen libraries der heutigen Übung. Standardmässig verwenden wir für `numpy`, `pandas` und `matplotlib` Abkürzungen.

In [32]:
import os
import fiona
import matplotlib.pyplot as plt
import geopandas
from shapely.geometry import MultiLineString

Danach definieren wir zwei dictionaries, die wir später zum Nachschlagen der Werte brauchen werden. 

In [33]:
verlauf_entries = {
    100: 'orberirdisch',
    200: 'unterirdisch bestimmt',
    300: 'unterirdisch unbestimmt'
}

stufe_entries = {
    -2: 'unterirdisch 2. Stufe',
    -1: 'unterirdisch 1. Stufe',
    0: 'ebenerdig 2. Stufe',
    1: 'überirdisch oder schwebend 1. Stufe',
    2: 'überirdisch oder schwebend 2. Stufe'
}

### Zusatzfunktion zum Plotten der Resultate

In [39]:
def show_lines():
    """
    Plots the map with the canton of Zurich and the rivers
    :return:
    """

    # Plot the map with the communalties
    communalities = geopandas.read_file('data\\gemeindegrenzen_kanton_zuerich.json')
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_aspect('equal')
    communalities.plot(ax=ax, color='white', edgecolor='black')

    # Plot the map with the rivers
    rivers = geopandas.read_file('data\\fliessgewaesser_kanton_zuerich_new.json')
    rivers.plot(ax=ax, color='red')    

    # Show the whole plot and return
    plt.draw()
    return

### Hauptfunktion: Lösen Sie diese Aufgaben

Die folgende Funktion kopiert die Inhalte einer Datei in eine andere. Dabei wird sichergestellt, dass der Geometrietyp in jedem Fall ein MultiLineString sein wird. Wir tun das deshalb, da es vorkommen kann, dass Esri Shapefiles gemischte Feature-Container aufweisen, die sowohl LineStrings als auch MultiLineStings enthalten können. Der Trick dabei: Wir bestimmen für jede Iteration den jeweiligen Geometrietyp und für den Fall, dass dieser ein LineString wäre, packen wir den in eine weitere Liste. Damit wird erfüllt, dass ein MultiLineString-Objekt im Gegensatz zum LineString-Objekt eine Listenebene mehr aufweist.  

1. Gehen Sie den Code durch und versuchen Sie, jeden Schritt nachzuvollziehen. 
2. Berechnen Sie die Länge jedes Abschnitts, indem Sie zuerst aus der Koordinatenliste ein MultiLineString-Objekt erstellen und dann eine geeignete Methode zur Bestimmung der Länge anwenden (siehe https://shapely.readthedocs.io/en/stable/manual.html). Speichern Sie diesen Wert in das neue Attrbut `SECTION_LENGTH`. 
3. Schlagen Sie die zu den Attributen `VERLAUF` und `STUFE` in den jeweiligen dictionaries die textuelle Bezeichnung nach und speichern Sie diesen Wert in die neuen Attribute `VERLAUF_DESC` und `STUFE_DESC`. 

Zusatzaufgabe:
4. Sie sollten merken, dass die Koordinaten dreidimensional (X, Y, Z) angegeben sind. Extrahieren Sie aus den Koordinatenlisten jeweils nur die X- und die Y-Koordinate und speichern Sie dies in das neue File. Diese Aufgabe ist ein bisschen tricky, das Sie dazu die Geometriedatentypen verstehen müssen – insbesondere, wie die Listen mit den Koordinaten verschachtelt sind. 

In [38]:
# Define the input and the output file
in_file = os.path.join(os.path.abspath(''), 'data', 'fliessgewaesser_kanton_zuerich.json')
out_file = os.path.join(os.path.abspath(''), 'data', 'fliessgewaesser_kanton_zuerich_new.json')

# Open the source file
with fiona.open(in_file) as src:

    # Determine the CRS, the driver, and copy the schema (because dictionaries are mutable).
    crs = src.crs
    driver = src.driver
    schema = src.schema.copy()

    # Hint for another time: We assert that the geometry type MUST be set to MultiLineString. The reason is that
    # Esri Shapefiles can contain different geometry types in one container, whereas we must be super clear.
    schema['geometry'] = 'MultiLineString'

    ################################################
    #
    # MODIFY THE SCHEMA
    #
    # a) add the length of each section to the new attribute 'SECTION_LENGTH'
    # b) add the description of the existing attribute 'VERLAUF' to the new attribute 'VERLAUF_DESC'
    # c) add the description of the existing attribute 'STUFE' to the new attribute 'STUFE_DESC'
    #
    ################################################

    # Open the file that should be written and iterate through the entries
    with fiona.open(out_file, 'w', driver=driver, schema=schema, crs=crs, encoding='utf-8') as tgt:
        for f in src:

            # Determine the type for each entry and the coordinates
            geom_type = f['geometry']['type']
            coords = f['geometry']['coordinates']

            # As it can occur that the input geometry object is a LineString or a MultiLineString, we must
            # assert that the dimensionality stays the same for both cases. Thus, put the coordinates into a
            # further list, if the geometry type should be a LineString
            if geom_type == 'LineString':
                coords = [coords]

            ################################################
            #
            # DETERMINE THE VALUES THAT SHOULD BE WRITTEN INTO THE ATTRIBUTES
            #
            ################################################

            # Overwrite the type and the coordinates with the new entries
            f['geometry']['type'] = 'MultiLineString'
            f['geometry']['coordinates'] = coords

            ################################################
            #
            # WRITE THE VALUES TO THE ACCORDING DICTIONARY ENTRY
            #
            ################################################

            tgt.write(f)

# Plot the results
show_lines()
print('Done.')

# Erkenntnisse

Mit grosser Wahrscheinlichkeit beinhaltete die heutige Übung viel Neues für Sie. Welche Parameter konnten Sie modellieren, welche nicht? Notieren Sie im anschliessenden Feld Ihre wichtigsten Erkenntnisse von heute:

*[Ihre Notizen]*

# Projekt
Arbeiten Sie am Projekt weiter und wenden Sie das, was Sie heute gelernt haben, auf Ihr Projekt an. Bereiten Sie die Daten so vor, dass wir nächste Woche mit der vektorbasierten Datenverarbeitung beginnen können. Bestimmen Sie zudem alle Referenzsysteme Ihrer Daten – es kann nämlich vorkommen, dass die Daten in unterschiedlichen Referenzsystemen gespeichert sind. 